<a href="https://colab.research.google.com/github/sonialouise/ts_class/blob/main/notebooks/SpontaneousActivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spontaneous Neuronal Activity Classifier
This spontaneous neuronal activity classifier has been trained to detect whether or not neuronal frequency recordings indicate neuronal activity. The data used to train this model was 151 seconds at a sampling rate of 3.65Hz. 

This notebook enables users to:
- Use our trained activity classifier on your own data
- Use our labelled data to retrain the activity classifier for different recording durations (greater or less than 151 seconds)
- Retrain the classifier on your own labelled data

To run each cell of the notebook, click the play arrow to the left. Alternatively, you can use shift + enter. Cells 1-3 are required if testing or training the model.

In [1]:
#@markdown The Github repository with all custom functions, the model training datasets and the pretrained models are copied to the environment of this notebook. This will take a couple of seconds.

#@markdown *Note: You can check the code underlying each cell by double-clicking on it.*

import os

# If in Colab and not yet downloaded, download GitHub repository and change working directory
if os.getcwd() == '/content':  
    !git clone https://github.com/sonialouise/ts_class.git
    os.chdir('ts_class')
    
# If executed as jupyter notebook on own computer, change to parent directory for imports
if os.path.basename( os.getcwd() ) == 'activity_classifier':
    %cd ..
    print('New working directory:', os.getcwd() )

fatal: destination path 'ts_class' already exists and is not an empty directory.


In [9]:
#@markdown Run this cell to pull the latest version of the repository


!git pull 

Already up to date.


In [3]:
#@markdown Download packages required to run the model

%%capture
!pip install -r requirements.txt

# python imports
import pickle
import argparse
import logging
import pandas as pd

# ts_class packages, imported from the downloaded Github repository
from activity_classifier.main import run_model
from activity_classifier.config import TSF_MODEL, RISE_MODEL, OBS, PREDICTION, OUTPUT_PATH
from activity_classifier.prepare_data import prepare_data
from activity_classifier.retrain_models import retrain_tsf, retrain_rise

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

# Running the classifier

In [4]:
#@markdown If you are testing the script, leave the below field unchanged. If you want to apply the model to your own data, you need to upload your data first. 

#@markdown To upload data, click on the **folder symbol ("Files")** on the left side of the Colaboratory notebook, and select 'Upload to session storage'. 

#@markdown Note that for data longer than 151 seconds, only the first 151 seconds will be used for the prediction, unless the model is retrained with the longer duration.

#@markdown Please enter path to your test data in Colab files

file_path = "data/test_data.csv" #@param {type:"string"}


In [5]:
#@markdown If you have retrained your model, please enter the new duration (seconds) and sampling rate (Hz) variables below (these should be exactly the same as used during retraining). Otherwise, leave the default values.

duration = 151 #@param {type:"number"}  

sampling_rate = 3.65 #@param {type: "number"}

In [6]:
#@markdown Play this cell to run the model on your data. The model predictions and the interpolated data can be found in the "Files" folder under ts_class -> data.

logging.info("1. Reading csv file...")
data = pd.read_csv(file_path, header=0)
logging.info("2. Normalising data...")
total_frames = int(duration * sampling_rate)
trace_data = data.iloc[:, 0:total_frames]
data = prepare_data(trace_data, duration, sampling_rate)
logging.info("3. Predicting with TimeSeries Forest Classifier...")
data = run_model(data, TSF_MODEL, 'TSF')
logging.info("4. Predicting with Random Interval Spectral Ensemble...")
data = run_model(data, RISE_MODEL, 'RISE')
logging.info("5. Saving output...")
data.to_csv(OUTPUT_PATH)
logging.info("6. Process complete")

/content/ts_class/activity_classifier/prepare_data.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[OBS] = [pd.Series(interpolate_data(row, seconds, end_frame_rate)) for row in np.array(data)]
/content/ts_class/activity_classifier/main.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f"{model_name}_{PREDICTION}"] = result


# Model Retraining
If your setup uses frequency recordings greater or less than 151 seconds, model retraining is required. To you this, you can either use our labelled data or upload your own.

If using your own data, the format is important. Data should be in a .csv file, with the column names indicating the frame number. The final column should be named 'status' and contain the activity label: 'inactive' or 'active'

|0|1|2|3|...|status|
|-|-|-|-|---|------|
|0.134|0.224|0.123|0.444|...|active
|0.005|0.002|0.000|0.001|...|inactive

In [5]:
#@markdown If you want to retrain on our data, do not adjust the below file path (but still play this cell)

#@markdown To upload your own data, click on the **folder symbol ("Files")** on the left side of the Colaboratory notebook, and select 'Upload to session storage'. Next, indicate the file path of the uploaded file in the variable **`training_file_path`** below. 

training_file_path = "data/training_data.csv" #@param {type:"string"}





In [ ]:
#@markdown If you want to retrain the model on our data, do not adjust the below sampling rate (but still play this cell)

#@markdown If you are using your own data, enter the recording sampling rate (Hz)

sampling_rate = 3.65 #@param {type: "number"}

In [10]:
#@markdown If you want to retrain the model on our data, update the duration field below per your requirements. Please note, the maximum duration available in our data is 301 seconds. For a longer duration, your own labelled data is required. 

#@markdown If you are using your own data, enter the total recording duration in seconds

duration = 151 #@param {type:"number"}  

In [6]:
#@markdown Play this cell to retrain the model. This should take approx 5 minutes. <br />
   
logging.info("1. Reading csv file...")
data = pd.read_csv(training_file_path, header=0)
logging.info("2. Normalising data...")
total_frames = int(duration * sampling_rate)
trace_data = data.iloc[:, 0:total_frames]
data = pd.concat([prepare_data(trace_data, duration, sampling_rate), data['status']], axis=1)
logging.info("3. Retraining Time Series Classifier...")
retrain_tsf(data)
logging.info("4. Retraining Random Interval Spectral Ensemble...")
retrain_rise(data)
logging.info("5. Retraining Complete")

RESULTS TSF: {'ACCURACY': 0.9059664824076512, 'PRECISION': 0.78944154209215, 'RECALL': 0.7456984496037586}
RESULTS RISE: {'ACCURACY': 0.9663741920388406, 'PRECISION': 0.9517366486824616, 'RECALL': 0.8822005378001917}
